In [ ]:

# -------------------------------------------------------------------------------------------------------------------
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

# 예측된 2024년부터 2075년까지의 데이터 불러오기
house_price_2024_2075 = pd.read_csv('예측된_집값_2024_2075.csv')
feat_sal_2024_2075 = pd.read_csv('예측된_임금_총액_2024_2075.csv')
education_expense_2024_2075 = pd.read_csv('예측된_사교육비_총액_2024_2075.csv')
fertile_women_2024_2075 = pd.read_csv('예측된_가임_여성수_2024_2075.csv')
marriage_count_2024_2075 = pd.read_csv('예측된_혼인건수_2024_2075.csv')

# 공통된 연도 (2024년부터 2075년까지)
years_to_predict_future = np.arange(2024, 2076)

# 공통적인 도시 목록
cities = house_price_2024_2075['City'].values

# 결과를 저장할 빈 데이터프레임 초기화
predictions_df = pd.DataFrame(columns=['City'] + years_to_predict_future.astype(str).tolist())

# 기존 학습된 모델을 사용하여 각 도시별로 예측을 수행
future_predictions = {}

for city in cities:
    print(f"예측을 위한 도시: {city}")

   # 기존 데이터를 사용하여 해당 도시의 학습된 모델을 불러옵니다.
    # 특성 행렬(X)와 목표 변수 벡터(y) 설정 (지역별로)
    X = pd.DataFrame({
        'house_price': house_price.loc[city],
        'wage': feat_sal.loc[city],
        'education_expense': education_expense.loc[city],
        'fertile_women': fertile_women.loc[city],
        'marriage_count': marriage_count.loc[city]
    })

    y = birth_rate.loc[city]

    # Ensure X and y have the same number of samples
    # This assumes 'birth_rate' has the correct index for alignment
    X = X.loc[y.index]  # Align X with y's index

    # 데이터 표준화 -> StandardScaler로 변경
    scaler = StandardScaler()

    # Impute NaN values with the mean before scaling
    imputer = SimpleImputer(strategy='mean') # Create an imputer instance
    X_imputed = imputer.fit_transform(X) # Impute NaN values in X

    X_scaled = scaler.fit_transform(X_imputed) # Scale the imputed data


    # PolynomialFeatures 차수 감소 (degree=1로 변경하여 모델 복잡도를 낮춤)
    poly = PolynomialFeatures(degree=1, include_bias=False)  # degree=1로 변경
    X_poly = poly.fit_transform(X_scaled)

    # Ridge 모델 하이퍼파라미터 튜닝 (GridSearchCV)
    param_grid = {
        'alpha': np.logspace(-4, 1, 6)  # alpha 값을 조금 더 크게 설정하여 규제 강화를 시도
    }

    grid_search = GridSearchCV(Ridge(max_iter=10000, random_state=42), param_grid, cv=5, scoring='r2')
    grid_search.fit(X_poly, y)

    # 훈련된 모델을 사용하여 2024-2075년 예측
    best_model = grid_search.best_estimator_

    # 예측할 미래 데이터 준비
    X_future = pd.DataFrame({
        'house_price': house_price_2024_2075.loc[house_price_2024_2075['City'] == city, years_to_predict_future.astype(str)].values.flatten(),
        'wage': feat_sal_2024_2075.loc[feat_sal_2024_2075['City'] == city, years_to_predict_future.astype(str)].values.flatten(),
        'education_expense': education_expense_2024_2075.loc[education_expense_2024_2075['City'] == city, years_to_predict_future.astype(str)].values.flatten(),
        'fertile_women': fertile_women_2024_2075.loc[fertile_women_2024_2075['City'] == city, years_to_predict_future.astype(str)].values.flatten(),
        'marriage_count': marriage_count_2024_2075.loc[marriage_count_2024_2075['City'] == city, years_to_predict_future.astype(str)].values.flatten()
    })

    # Impute NaN values in X_future as well
    X_future_imputed = imputer.transform(X_future)  # Use the same imputer fitted earlier

    # X_future 데이터 표준화
    X_future_scaled = scaler.transform(X_future_imputed) # Scale the imputed future data

    # PolynomialFeatures 적용 (훈련 시 사용한 것과 동일하게 변환)
    X_future_poly = poly.transform(X_future_scaled)

    # 예측값 계산
    future_predictions_city = best_model.predict(X_future_poly)

    # 예측값이 0보다 작은 경우는 0으로 설정
    future_predictions_city = np.maximum(future_predictions_city, 0)  # 출생아수는 0보다 커야 하므로 0으로 설정

    # 변화폭을 더 줄이기 위해 급격한 변화가 없도록 예측값의 변화를 조정
    future_predictions_city = np.clip(future_predictions_city, future_predictions_city.min(), future_predictions_city.max())  # 예측값이 급격히 변하지 않도록 제한

    # 결과 저장
    future_predictions[city] = future_predictions_city

    # 예측된 신생아수 데이터를 DataFrame으로 변환하여 predictions_df에 추가
    predictions_df.loc[len(predictions_df)] = [city] + future_predictions_city.tolist()

# 예측된 데이터를 '예측된_신생아수_2024_2075.csv' 파일로 저장
predictions_df.to_csv('예측된_신생아수_2024_2075.csv', index=False)

# 결과 출력
print("\n2024년부터 2075년까지 각 도시별 예측된 출생아수가 '예측된_신생아수_2024_2075.csv'에 저장되었습니다.")
